In [1]:
import json
import numpy
import random

In [2]:
from deap import base
from deap import algorithms
from deap import creator
from deap import tools

In [3]:
with open("4.txt", "r") as f:
    source = f.read()

separated_values = []
separated_rows = source.split("\n")

for row in separated_rows:
    separated_values.append(list(map(lambda x:float(x), row.split(' '))))
    
MAX_WEIGHT, MAX_VOLUME = separated_values[0]
data = separated_values[1:]
ITEM_COUNT = len(data)

print('Max weight:', MAX_WEIGHT,'\nMax volume:', MAX_VOLUME, '\nItem count:', ITEM_COUNT)

Max weight: 13000.0 
Max volume: 12.0 
Item count: 30


In [4]:
creator.create("Fitness", base.Fitness, weights=(-1.0,-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)
toolbox = base.Toolbox()

items = {}
for i in range(ITEM_COUNT):
    items[i] = (data[i][0], data[i][1], data[i][2])
items

## Attribute generator
toolbox.register("attr_item", random.randrange, ITEM_COUNT)
## Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, ITEM_COUNT)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [5]:
def evalKnapsack(individual):
    weight = 0.0
    volume = 0.0
    cost = 0.0
    for item in individual:
        weight += items[item][0]
        volume += items[item][1]
        cost += items[item][2]
    if weight > MAX_WEIGHT or volume > MAX_VOLUME:
        return MAX_WEIGHT, MAX_VOLUME,  0           
    return weight, volume, cost

In [6]:
def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

def mutSet(individual):
    individual.add(random.randrange(ITEM_COUNT))
    return individual,

In [7]:
toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

def main():
    NGEN = 500
    MU = 200
    LAMBDA = 100
    CXPB = 0.2
    MUTPB = 0.1
    
    pop = toolbox.population(n=MU)
    stats = tools.Statistics(lambda val: val)
    stats.register("max", numpy.max, axis=0)
    
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats)
    
    return pop

In [8]:
result = main()[-1]

gen	nevals	max                                                                               
0  	200   	Individual({2, 4, 5, 7, 8, 9, 11, 12, 13, 15, 16, 17, 19, 20, 22, 26, 27, 28, 29})
1  	38    	Individual({3, 6, 7, 8, 11, 13, 14, 15, 16, 18, 19, 21, 22, 23, 24, 26, 27, 29})  
2  	38    	Individual({2, 3, 4, 6, 8, 9, 10, 11, 12, 13, 17, 18, 20, 22, 23, 24, 25, 28, 29})
3  	30    	Individual({0, 1, 2, 3, 5, 6, 8, 9, 11, 13, 14, 15, 18, 19, 20, 21, 22, 24, 25, 26, 27, 29})
4  	30    	Individual({0, 3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 25, 26, 27, 29})
5  	33    	Individual({1, 3, 4, 6, 8, 9, 10, 11, 13, 18, 19, 21, 22, 25, 26, 27, 28, 29})               
6  	26    	Individual({4, 5, 7, 9, 13, 16, 22, 23, 25, 26, 27, 28})                                     
7  	29    	Individual({1, 4, 7, 9, 14, 20, 21, 23, 24, 26, 27, 29})                                     
8  	24    	Individual({0, 3, 5, 7, 14, 20, 21, 23, 24, 25, 26, 28})                                 

In [9]:
for item in result:
    print(item, data[item])
RESULT_WEIGHT, RESULT_VOLUME, RESULT_COST = evalKnapsack(result)
print('Result weight: [{} / {}]\nResult volume: [{} / {}]\nResult cost: [{}]'.format(RESULT_WEIGHT, MAX_WEIGHT, RESULT_VOLUME, MAX_VOLUME, RESULT_COST))

1 [659.0, 0.8, 165.0]
3 [1179.0, 0.9, 366.0]
6 [711.0, 1.1, 281.0]
9 [534.0, 1.0, 340.0]
10 [786.0, 0.9, 156.0]
11 [885.0, 0.8, 240.0]
16 [1006.0, 1.0, 182.0]
17 [1580.0, 0.9, 248.0]
18 [255.0, 0.7, 151.0]
20 [405.0, 0.5, 113.0]
22 [775.0, 0.7, 247.0]
23 [637.0, 0.6, 226.0]
25 [1253.0, 0.6, 343.0]
27 [1596.0, 0.5, 174.0]
29 [577.0, 0.9, 364.0]
Result weight: [12838.0 / 13000.0]
Result volume: [11.899999999999999 / 12.0]
Result cost: [3596.0]


In [10]:
results = {
    'Cost' : RESULT_COST,
    'Weight' : RESULT_WEIGHT,
    'Volume' : RESULT_VOLUME    
}
with open('4.1_result.json', 'w') as file:
    json.dump(results, file, indent=10)